In [1]:
import warnings; warnings.simplefilter('ignore')
import logging
from kiteconnect import KiteConnect
logging.basicConfig(level=logging.ERROR)
import ssl
from requests.exceptions import ConnectionError
from http.client import RemoteDisconnected
from urllib3.exceptions import ProtocolError
from ssl import SSLError
import math
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Dense, LSTM,Dropout,Input,Flatten,Activation,LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard 
import tensorflow.keras
import matplotlib.pyplot as plt
import nsepy
import pprint
import xgboost
import os
from datetime import date,timedelta
import datetime
import time
import mplcursors
#plt.style.use('fivethirtyeight')
from tqdm import tqdm
import pickle
from sklearn import preprocessing
import livelossplot
import wandb
# from wandb.keras import WandbCallback
from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from IPython.display import clear_output
import seaborn as sns
import cv2
import threading
plt.style.use("seaborn")
import glob
import pandas as pd
import autokeras as ak
homediraddr='/home/hemangjoshi37a_gmail_com'
import pandas as pd
from ta import add_all_ta_features
from ta.utils import dropna
from sklearn.model_selection import GridSearchCV

In [8]:
mydata = joblib.load('/home/hemangjoshi37a_gmail_com/pfiles/USDINR20OCTFUT_DF.p')
mydata['hour'] = mydata['date'].dt.hour
mydata['minute'] = mydata['date'].dt.minute
mydata['dayofweek'] = mydata['date'].dt.dayofweek
mydata['quarter'] = mydata['date'].dt.quarter
mydata['month'] = mydata['date'].dt.month
mydata['year'] = mydata['date'].dt.year
mydata['dayofyear'] = mydata['date'].dt.dayofyear
mydata['dayofmonth'] = mydata['date'].dt.day
mydata['weekofyear'] = mydata['date'].dt.weekofyear
dates = mydata['date']
mydata = mydata.drop(columns=['date'])
mydata['pred_close'] = mydata['close']
mydata['pred_high'] = mydata['high']
mydata['pred_open'] = mydata['open']
mydata['pred_low'] = mydata['low']

mydata = mydata.fillna(0)
mydata = add_all_ta_features(
    mydata, open="open", high="high", low="low", close="close", volume="volume")
mydata.replace([np.inf, -np.inf], np.nan)
mydata = mydata.drop(columns=['others_cr'])
mydata = mydata.fillna(0)

In [9]:
for i in range(101049):
    mydata['pred_close'][i] = mydata['close'][i+1]
    
train = mydata.iloc[110:95999, :]
test = mydata.iloc[95999:, :]

xtrain = train.drop(columns=['pred_close','pred_high','pred_low','pred_open']).to_numpy()
ytrain_close = train['pred_close'].to_numpy()

xtest = test.drop(columns=['pred_close','pred_high','pred_low','pred_open']).to_numpy()
ytest_close = test['pred_close'].to_numpy()

In [10]:
train

,open,high,low,close,volume,hour,minute,dayofweek,quarter,month,...,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,others_dr,others_dlr
110,73.5400,73.5400,73.5400,73.5400,0,9,22,2,4,10,...,99.597306,100.000000,100.000000,100.000000,-0.000000,0.116213,0.0,0.194148,0.000000,0.000000
111,73.5400,73.5400,73.5400,73.5400,0,9,23,2,4,10,...,99.595904,100.000000,100.000000,100.000000,-0.000000,0.112978,0.0,0.194148,0.000000,0.000000
112,73.5400,73.5400,73.5400,73.5400,0,9,24,2,4,10,...,99.594616,100.000000,100.000000,100.000000,-0.000000,0.110037,0.0,0.194148,0.000000,0.000000
113,73.5400,73.5400,73.5400,73.5400,0,9,25,2,4,10,...,99.593433,0.000000,100.000000,100.000000,-0.000000,0.107390,0.0,0.194148,0.000000,0.000000
114,73.5400,73.5400,73.5350,73.5350,2,9,26,2,4,10,...,99.565760,41.404358,96.491228,98.830409,-3.508772,0.104316,0.0,0.183924,-0.006799,-0.006799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95994,73.8275,73.8375,73.8275,73.8350,207,12,42,4,3,9,...,10.265829,64.389003,94.117647,77.915377,-5.882353,0.013559,0.0,0.003386,0.010159,0.010158
95995,73.8325,73.8475,73.8325,73.8450,348,12,43,4,3,9,...,13.000836,66.609942,93.333333,87.973856,-6.666667,0.016912,0.0,0.016930,0.013544,0.013543
95996,73.8450,73.8450,73.8350,73.8350,210,12,44,4,3,9,...,13.303837,61.124643,64.285714,83.912232,-35.714286,0.019824,0.0,0.016933,-0.013542,-0.013543
95997,73.8350,73.8525,73.8350,73.8525,238,12,45,4,3,9,...,15.885944,65.238189,100.000000,85.873016,-0.000000,0.022838,0.0,0.033863,0.023701,0.023699


In [ ]:
input_node = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(inputs=input_node, outputs=output_node, max_trials=100,
                   overwrite=True)
reg.fit(xtrain, ytrain_close, epochs=10)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
dense_block_1/n...|2                 |?                 
dense_block_1/u...|False             |?                 
dense_block_1/d...|0.5               |?                 
dense_block_1/u...|32                |?                 
dense_block_1/u...|32                |?                 
regression_head...|0                 |?                 
optimizer         |adam              |?                 
learning_rate     |0.001             |?                 



In [ ]:
model = reg.export_model()
model.summary()
ypredicted =  model.predict(xtest.astype(np.unicode))
ypredicted

In [ ]:
model.predict(xtest.astype(np.unicode))

In [ ]:
import joblib
joblib.dump(reg,'autokeras.p')

In [ ]:
ytest_closedf =  pd.DataFrame(ytest_close)
predicteddf =  pd.DataFrame(ypredicted)
newdfdf= pd.concat([ytest_closedf, predicteddf], axis=1, sort=False)
#newdfdf[''].plot()
newdfdf

In [ ]:
for i in range(101049):
    mydata['pred_close'][i] = mydata['close'][i+1]
    mydata['pred_open'][i] = mydata['open'][i+1]
    mydata['pred_high'][i] = mydata['high'][i+1]
    mydata['pred_low'][i] = mydata['low'][i+1]
    
mydata = mydata.iloc[100:,:]
train = mydata.iloc[:95999, :]
test = mydata.iloc[95999:, :]

In [ ]:
xtrain = train.drop(columns=['pred_close','pred_high','pred_low','pred_open']).to_numpy()
ytrain_close = train['pred_close'].to_numpy()
ytrain_open = train['pred_open'].to_numpy()
ytrain_high = train['pred_high'].to_numpy()
ytrain_low = train['pred_low'].to_numpy()
xtest = test.drop(columns=['pred_close','pred_high','pred_low','pred_open']).to_numpy()
ytest_close = test['pred_close'].to_numpy()
ytest_open = test['pred_open'].to_numpy()
ytest_high = test['pred_high'].to_numpy()
ytest_low = test['pred_low'].to_numpy()

print('xtrain shape : ' + str(xtrain.shape))
print('ytrain shape : ' + str(ytrain_close.shape))
print('xtest shape : ' + str(xtest.shape))
print('ytest shape : ' + str(ytest_close.shape))

In [ ]:
nr=xgboost.XGBRegressor(
    
    colsample_bytree= 0.7, 
    learning_rate= 0.09, max_depth= 20, 
    min_child_weight=4,
    n_estimators=400, 
    nthread=4, 
    objective='reg:linear',
    silent= 1, 
    subsample=0.5)
 


model_close=nr
model_open=nr
model_high=nr
model_low=nr

eval_set_close=[(xtrain,ytrain_close),(xtest,ytest_close)]
eval_set_open=[(xtrain,ytrain_open),(xtest,ytest_open)]
eval_set_high=[(xtrain,ytrain_high),(xtest,ytest_high)]
eval_set_low=[(xtrain,ytrain_low),(xtest,ytest_low)]

   
                 colsample_bytree=0.6,
                 gamma=0.6,
                 learning_rate=0.005,
                 max_depth=50,
                 min_child_weight=0.5,
                 n_estimators=700,
                 gpu_id=0,
                 tree_method='gpu_hist',
                 objective='reg:squarederror',
                 reg_alpha=0.85,
                 reg_lambda=0.65,
                 subsample=0.8,
                 seed=30,
                 verbosity=1,
                 predictor = 'gpu_predictor',
                 #n_jobs=-1,

In [ ]:
model_close.fit(xtrain,ytrain_close,eval_metric=["error", "logloss"],eval_set=eval_set_open,verbose=True)
model_open.fit(xtrain,ytrain_open,eval_metric=["error", "logloss"],eval_set=eval_set_open,verbose=True)
model_high.fit(xtrain,ytrain_high,eval_metric=["error", "logloss"],eval_set=eval_set_high,verbose=True)
model_low.fit(xtrain,ytrain_low,eval_metric=["error", "logloss"],eval_set=eval_set_low,verbose=True)

joblib.dump(model_close, "xgbregressor19_close.p")
joblib.dump(model_open, "xgbregressor19_open.p")
joblib.dump(model_high, "xgbregressor19_high.p")
joblib.dump(model_low, "xgbregressor19_low.p")

In [ ]:
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear','reg:squarederror'],
              'learning_rate': [.03, 0.05, .07,0.09], #so called `eta` value
              'max_depth': [5, 6, 7,20],
              'min_child_weight': [4,6,8],
              'silent': [1],
              'subsample': [0.7,0.5],
              'colsample_bytree': [0.7],
              'n_estimators': [500,400,600]}

xgb_grid = GridSearchCV(model_close,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(xtrain,
         ytrain_close)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
parameters = {
              'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [0.09,0.1,0.15], #so called `eta` value
              'max_depth': [20,30,40],
              'min_child_weight': [2,3,4],
              'silent': [1],
              'subsample': [0.3,0.4,0.5],
              'colsample_bytree': [0.6,0.7,0.8],
              'n_estimators': [500,400,600]}

xgb_grid = GridSearchCV(model_close,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(xtrain,
         ytrain_close)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
test_done = test
test_done['pred_done_close'] = test['pred_close']
test_done['pred_done_open'] = test['pred_open']
test_done['pred_done_high'] = test['pred_high']
test_done['pred_done_low'] = test['pred_low']

for i in range(96100,101051,1):
    test_done['pred_done_close'][i] = model_close.predict(test.drop(columns=['pred_close','pred_high','pred_open','pred_low','pred_done_close','pred_done_high','pred_done_open','pred_done_low']).loc[i].to_numpy().reshape((1,86)))
    test_done['pred_done_open'][i] = model_open.predict(test.drop(columns=['pred_close','pred_high','pred_open','pred_low','pred_done_close','pred_done_high','pred_done_open','pred_done_low']).loc[i].to_numpy().reshape((1,86)))
    test_done['pred_done_high'][i] = model_high.predict(test.drop(columns=['pred_close','pred_high','pred_open','pred_low','pred_done_close','pred_done_high','pred_done_open','pred_done_low']).loc[i].to_numpy().reshape((1,86)))
    test_done['pred_done_low'][i] = model_low.predict(test.drop(columns=['pred_close','pred_high','pred_open','pred_low','pred_done_close','pred_done_high','pred_done_open','pred_done_low']).loc[i].to_numpy().reshape((1,86)))
    
test_done['pred_diff_close'] = test_done['pred_done_close']
test_done['pred_diff_open'] = test_done['pred_done_open']
test_done['pred_diff_high'] = test_done['pred_done_high']
test_done['pred_diff_low'] = test_done['pred_done_low']

test_done['pred_diff_close'] = test_done['pred_close'] - test_done['pred_done_close']
test_done['pred_diff_open'] = test_done['pred_open'] - test_done['pred_done_open']
test_done['pred_diff_high'] = test_done['pred_high'] - test_done['pred_done_high']
test_done['pred_diff_low'] = test_done['pred_low'] - test_done['pred_done_low']

test_done = test_done.drop([101051,96099])
test_done

In [ ]:
test_done['pred_diff_close'] = test_done['pred_diff_close'].abs()
test_done['pred_diff_close'].describe()

In [ ]:
test_done['pred_diff_open'] = test_done['pred_diff_open'].abs()
test_done['pred_diff_open'].describe()

In [ ]:
test_done.iloc[500:800,14:16].plot(figsize=(15,7))

In [ ]:
rec_test_done = test_done
#test = test.drop(columns=['pred_high','pred_close'])
#rec_test_done = rec_test_done.drop(columns=['pred_high','pred_low','pred_open','pred_close',
#                                            'pred_done_close','pred_done_open','pred_done_high',
#                                            'pred_done_low','pred_diff_close','pred_diff_open',
#                                            'pred_diff_high','pred_diff_low'])  
for i in range(96100,101050,1):
  
    rec_test_done['pred_done_close'][i] = model_close.predict(rec_test_done.
                                                              drop(columns=['pred_high','pred_low','pred_open','pred_close',
                                            'pred_done_close','pred_done_open','pred_done_high',
                                            'pred_done_low','pred_diff_close','pred_diff_open',
                                                                            'pred_diff_high','pred_diff_low'])
                                                              .loc[i].to_numpy().reshape((1,14)))
    rec_test_done['pred_done_open'][i] = model_open.predict(rec_test_done.
                                                              drop(columns=['pred_high','pred_low','pred_open','pred_close',
                                            'pred_done_close','pred_done_open','pred_done_high',
                                            'pred_done_low','pred_diff_close','pred_diff_open',
                                            'pred_diff_high','pred_diff_low']).loc[i].to_numpy().reshape((1,14)))
    rec_test_done['pred_done_high'][i] = model_high.predict(rec_test_done.
                                                              drop(columns=['pred_high','pred_low','pred_open','pred_close',
                                            'pred_done_close','pred_done_open','pred_done_high',
                                            'pred_done_low','pred_diff_close','pred_diff_open',
                                            'pred_diff_high','pred_diff_low']).loc[i].to_numpy().reshape((1,14)))
    rec_test_done['pred_done_low'][i] = model_low.predict(rec_test_done.
                                                              drop(columns=['pred_high','pred_low','pred_open','pred_close',
                                            'pred_done_close','pred_done_open','pred_done_high',
                                            'pred_done_low','pred_diff_close','pred_diff_open',
                                            'pred_diff_high','pred_diff_low']).loc[i].to_numpy().reshape((1,14)))
    
    rec_test_done['close'][i+1] = rec_test_done['pred_done_close'][i]
    rec_test_done['open'][i+1] = rec_test_done['pred_done_open'][i]
    rec_test_done['high'][i+1] = rec_test_done['pred_done_high'][i]
    rec_test_done['low'][i+1] = rec_test_done['pred_done_low'][i]

rec_test_done

In [ ]:
rec_test_done['pred_diff_close'] = rec_test_done['pred_close']
rec_test_done['pred_diff_close'] = rec_test_done['pred_close'] - rec_test_done['pred_done_close']
#new_test_done = new_test_done.drop([101051,96099])
rec_test_done

In [ ]:
rec_test_done['pred_diff_close'].describe()

In [ ]:
rec_test_done.iloc[0:200,14:18].plot(figsize=(20,12))
rec_test_done.iloc[0:200,18:22].plot(figsize=(20,12))